# NIRISS SOSS Extraction - Reference File Creation.

In [1]:
import numpy as np

from astropy.io import fits, ascii

from SOSS.trace import tracepol
from SOSS.extract.Generate_ref_files.soss_ref_files import init_spec_trace, calc_2d_wave_map, init_wave_map, init_spec_profile, init_spec_kernel

## The 1D spectral trace reference files.

The 1D spectral trace reference files contain, as a function of wavelength, the x, y coordinates of the trace in pixels, the tilt angle in degrees and the throughput. In this exmaple we prepare data from various sources and use `init_spec_trace()` to create the reference file.

In [2]:
# We will use the following sources for this reference file. These can be modified as needed.
throughput_file = 'files/NIRISS_Throughput_STScI.fits'
tilt_file = 'files/SOSS_wavelength_dependent_tilt.ecsv'

In [3]:
# All sources will be modified to correspond to this wavelength grid.
wavemin = 0.5
wavemax = 5.5
nwave = 5001
wave_grid = np.linspace(wavemin, wavemax, nwave)

In [4]:
# Read the SOSS total throughput as a function of wavelength.
tab, hdr = fits.getdata(throughput_file, ext=1, header=True)

# Interpolate to the reference wavelength grid.
throughput = np.zeros((nwave, 3))
throughput[:, 0] = np.interp(wave_grid, tab[0]['LAMBDA']/1e3, tab[0]['SOSS_ORDER1'])
throughput[:, 1] = np.interp(wave_grid, tab[0]['LAMBDA']/1e3, tab[0]['SOSS_ORDER2'])
throughput[:, 2] = np.interp(wave_grid, tab[0]['LAMBDA']/1e3, tab[0]['SOSS_ORDER3'])

# Fix small negative throughput values.
throughput = np.where(throughput < 0, 0, throughput)

In [5]:
# Read the tilt as a function of wavelength.
tab = ascii.read(tilt_file)

# Interpolate the tilt to the same wavelengths as the throughput.
# Default bounds handling (constant boundary) is fine.
tilt = np.zeros((nwave, 3))
tilt[:, 0] = np.interp(wave_grid, tab['Wavelength'], tab['order 1'])
tilt[:, 1] = np.interp(wave_grid, tab['Wavelength'], tab['order 2'])
tilt[:, 2] = np.interp(wave_grid, tab['Wavelength'], tab['order 3'])

In [6]:
# Read the trace position parameters.
tracepars = tracepol.get_tracepars('../../trace/NIRISS_GR700_trace_extended.csv')

# Get positions for all orders and subarrays.
xtrace_order1, ytrace_order1, mask = tracepol.wavelength_to_pix(wave_grid, tracepars, m=1, frame='dms', subarray='FULL')
xtrace_order1 = np.where(mask, xtrace_order1, np.nan)
ytrace_order1 = np.where(mask, ytrace_order1, np.nan)

xtrace_order2, ytrace_order2, mask = tracepol.wavelength_to_pix(wave_grid, tracepars, m=2, frame='dms', subarray='FULL')
xtrace_order2 = np.where(mask, xtrace_order2, np.nan)
ytrace_order2 = np.where(mask, ytrace_order2, np.nan)

xtrace_order3, ytrace_order3, mask = tracepol.wavelength_to_pix(wave_grid, tracepars, m=3, frame='dms', subarray='FULL')
xtrace_order3 = np.where(mask, xtrace_order3, np.nan)
ytrace_order3 = np.where(mask, ytrace_order3, np.nan)

xtrace = np.zeros((nwave, 3))
xtrace[:, 0] = xtrace_order1
xtrace[:, 1] = xtrace_order2
xtrace[:, 2] = xtrace_order3

ytrace = np.zeros((nwave, 3))
ytrace[:, 0] = ytrace_order1
ytrace[:, 1] = ytrace_order2
ytrace[:, 2] = ytrace_order3

In [7]:
# Call init_spec_trace with the cleaned input data. This will perform checks on the input and built the fits file structure.
hdul = init_spec_trace(wave_grid, xtrace, ytrace, tilt, throughput, 'FULL')

In [8]:
# If necessary manual changes and additions can be made here, before saving the file.
filename = hdul[0].header['FILENAME']
hdul.writeto(filename, overwrite=True)
hdul.writeto(filename + '.gz', overwrite=True)

## The 2D wavelength map reference files.

The 2D wavelength map reference files contain the oversampled and padded wavelength maps of each order on the detector. In this example we use the 1D spectral trace reference for FULL subarray to compute the 2D wavelength maps for each order and use init_wave_map() to create the reference file.

In [9]:
# We will use the following sources for this reference file. These can be modified as needed.
trace_file = 'SOSS_ref_trace_table_FULL.fits'

In [10]:
padding = 10
oversample = 2

dimx = oversample*(2048 + 2*padding)
dimy = oversample*(2048 + 2*padding)

wave_map_2d = np.zeros((dimx, dimy, 3))

# Read the 1D trace reference file.
data = fits.getdata(trace_file, ext=1)

# Compute the 2D wavelength map.
wave_map_2d[:, :, 0] = calc_2d_wave_map(data['WAVELENGTH'], data['X'], data['Y'], data['TILT'], oversample=oversample, padding=padding)

# Read the 1D trace reference file.
data = fits.getdata(trace_file, ext=2)

# Compute the 2D wavelength map.
wave_map_2d[:, :, 1] = calc_2d_wave_map(data['WAVELENGTH'], data['X'], data['Y'], data['TILT'], oversample=oversample, padding=padding)

# Read the 1D trace reference file.
data = fits.getdata(trace_file, ext=3)

# Compute the 2D wavelength map.
wave_map_2d[:, :, 2] = calc_2d_wave_map(data['WAVELENGTH'], data['X'], data['Y'], data['TILT'], oversample=oversample, padding=padding)

In [11]:
hdul = init_wave_map(wave_map_2d, oversample, padding, 'SUBSTRIP256')

In [12]:
# If necessary manual changes and additions can be made here, before saving the file.
filename = hdul[0].header['FILENAME']
hdul.writeto(filename, overwrite=True)
hdul.writeto(filename + '.gz', overwrite=True)

## The 2D spectral profile reference files.

The 2D spectral profile reference files contain the oversampled and padded normalized? PSF? of each order on the detector. In this example we use an input file generated by Loïc from a simulated SOSS observation and use init_spec_profile() to create the reference file.

In [13]:
# We will use the following sources for this reference file. These can be modified as needed.
profile_file = 'files/2DTrace.fits'

In [14]:
# Read the profile file provided by Loïc.
profile_2d = fits.getdata(profile_file, ext=0)
profile_2d = np.moveaxis(profile_2d, 0, -1)

# The padding and oversamspling used to generate the 2D profile. 
padding = 10
oversample = 2

# The provided file is for SUBSTRIP256, we pad this to the FULL subarray.
nrows, ncols, _ = profile_2d.shape
dimy = oversample*(2048 + 2*padding)
dimx = oversample*(2048 + 2*padding)

tmp = np.full((dimy, dimx, 3), fill_value=np.nan)
tmp[-nrows:] = profile_2d
profile_2d = tmp

In [15]:
# Call init_spec_profile with the prepared input data.
hdul = init_spec_profile(profile_2d, oversample, padding, 'SUBSTRIP256')

In [16]:
# If necessary manual changes and additions can be made here, before saving the file.
filename = hdul[0].header['FILENAME']
hdul.writeto(filename, overwrite=True)
hdul.writeto(filename + '.gz', overwrite=True)

## The spectral kernel reference file.

The spectral kernel reference file contains the SOSS convolution kernel as a function of wavelength. In this example we use an input file generated by Loïc using WebbPSF and use init_spec_profile() to create the reference file.

In [17]:
# We will use the following sources for this reference file. These can be modified as needed.
kernel_file = 'files/spectral_kernel_matrix_os_10_width_15pixels.fits'

In [18]:
# Read the kernel file provided by Loïc, this is based on WebbPSF.
kernels, hdr = fits.getdata(kernel_file, header=True)

specos = hdr['SPECOS']
halfwidth = hdr['HALFWIDT']
nwave = hdr['NWAVE']
wavemin = hdr['WAVE0']
wavemax = hdr['WAVEN']

# Build the wavelength array.
wavelengths = np.linspace(wavemin, wavemax, nwave)
wavelengths = np.ones_like(kernels) * wavelengths

In [19]:
# Call init_spec_kernel with the prepared input data.
hdul = init_spec_kernel(wavelengths, kernels, specos, halfwidth, nwave, wavemin, wavemax)

In [20]:
# If necessary manual changes and additions can be made here, before saving the file.
filename = hdul[0].header['FILENAME']
hdul.writeto(filename, overwrite=True)
hdul.writeto(filename + '.gz', overwrite=True)